In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

from tensorflow.keras.callbacks import *

from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

import os

2021-09-06 19:05:27.418452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-06 19:05:27.418652: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [23]:
frame = pd.read_csv(r'Predicting_Bike_Share_Demand_with_LSTM-master/hour.csv')
df = pd.DataFrame(frame)
col = list(df.columns.values)

df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [24]:
df = df.drop(['instant','dteday','casual','registered'],axis=1)

In [25]:
#get input and output
x = df.drop("cnt",axis=1)
y = df["cnt"]

In [26]:
#split data into train and test
x_train, x_test ,y_train , y_test = train_test_split(x,y,test_size=0.2,shuffle=False)

In [30]:
#normalise data
ct = make_column_transformer(
 (MinMaxScaler(),["weathersit"]) , remainder="passthrough"
 )


ct.fit(x_train)

x_train_normalized = ct.transform(x_train)

x_test_normalized = ct.transform(x_test)

x_train_normalized.shape

(13903, 12)

In [31]:
#create Model structure
with tf.device('/device:CPU:0'):
    model = Sequential()
    model.add(Dense(45, kernel_initializer='normal',  activation='relu', input_shape=(12,) ))    
    model.add(Dense(35, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(35, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(1, kernel_initializer='normal',activation=None))

In [11]:
gpus = tf.config.experimental.list_physical_devices('CPU')
print(gpus)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [32]:
#set model loss and optimizer
with tf.device('/device:CPU:0'):
    model.compile(optimizer= Adam(learning_rate=0.00001),loss='mae', metrics=['mae','mape'])

In [33]:
#checkpoint to save best Model which isnt overfitted
checkpoint = ModelCheckpoint('Model1', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min' )

In [40]:
#train Model
with tf.device('/device:CPU:0'):
    history = model.fit(x_train_normalized, y_train.values, epochs=10 , batch_size=10,shuffle=False   )

Epoch 1/10
1391/1391 [==============================] - 1s 1ms/step - loss: 108.6227 - mae: 108.6228 - mape: 342.7150
Epoch 2/10
1391/1391 [==============================] - 1s 1ms/step - loss: 108.4928 - mae: 108.4928 - mape: 340.7074
Epoch 3/10
1391/1391 [==============================] - 1s 1ms/step - loss: 108.3627 - mae: 108.3627 - mape: 338.7243
Epoch 4/10
1391/1391 [==============================] - 1s 1ms/step - loss: 108.2268 - mae: 108.2268 - mape: 336.8828
Epoch 5/10
1391/1391 [==============================] - 1s 1ms/step - loss: 108.1031 - mae: 108.1031 - mape: 335.1174
Epoch 6/10
1391/1391 [==============================] - 1s 1ms/step - loss: 107.9767 - mae: 107.9767 - mape: 333.2210
Epoch 7/10
1391/1391 [==============================] - 1s 1ms/step - loss: 107.8494 - mae: 107.8494 - mape: 331.2088
Epoch 8/10
1391/1391 [==============================] - 2s 1ms/step - loss: 107.7229 - mae: 107.7229 - mape: 329.1092
Epoch 9/10
1391/1391 [==============================] - 

In [39]:
model.evaluate(x_test_normalized,y_test.values)

109/109 [==============================] - 0s 750us/step - loss: 156.8631 - mae: 156.8631 - mape: 310.7234


[156.86306762695312, 156.86306762695312, 310.72344970703125]

In [37]:
#stationnamesframe =   df.drop_duplicates() 
#stationnamesframe = pd.DataFrame(stationnamesframe,columns=['station1'])
#stationnamesframe =   stationnamesframe.drop_duplicates() 
#stationnames =  pd.DataFrame(stationnamesframe , columns=['station1'])['station1'].to_numpy().tolist()
#print(len(stationnames))

In [22]:
#show result data linear


predict_train = model.predict(x_train_normalized)
y1 =  y_train 
y2 =  predict_train 
#test = pd.DataFrame({ 'y':y1 ,'y2':y2})#

test1 = df.iloc[:y_train.shape[0]]
test1 = pd.DataFrame(test1)
test1['y'] = y1
test1['y2'] = y2
#test1 = test1[test1['station1']==station]
test1['yearweek'] = pd.to_datetime(test1['year'].astype(str) + ' ' + test1['week'].astype(str) + ' 1',format='%Y %U %w')
#print(test1)

plt.plot(test1['yearweek'],test1['y'],label='real' , color = 'royalblue'  )
plt.plot(test1['yearweek'],test1['y2'],label='train' , color = 'orange'  )
predict_test = model.predict(x_test_normalized)
y1 =  y_test 
y2 =  predict_test 
#test = pd.DataFrame({ 'y':y1 ,'y2':y2})#
test2 = test.iloc[-y_test.shape[0]:]
test2 = pd.DataFrame(test2)
#print(test2)
test2['y'] = y1
test2['y2'] = y2
#test2 = test2[test2['station1']==station]
test2['yearweek'] = pd.to_datetime(test2['year'].astype(str) + ' ' + test2['week'].astype(str) + ' 1',format='%Y %U %w')
#print(test2)
#print(test2['y'])

try:
    mae = mean_absolute_error(test2['y'] ,test2['y2'] )
    plt.title(label=station+", mae: {:0.2f}".format(mae) ,
      fontsize=40,
      color="black")
    #plt.show()
except:
    plt.title(label=station+", mae: nan"  ,
      fontsize=40,
      color="black")
plt.plot(test2['yearweek'],test2['y'],label='_Hidden' , color = 'royalblue'  )
plt.plot(test2['yearweek'],test2['y2'],label='predict' , color = 'black' )

plt.legend(loc="upper left")
#plt.savefig( 'Model3Images/'+station+'linear.jpg',bbox_inches='tight', dpi=150)
plt.show()

KeyError: 'week'